In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Installing dependencies...")
!pip install -q torch torchvision torchaudio
!pip install -q transformers
!pip install -q optimum
!pip install -q onnx
!pip install -q accelerate
!pip install -q sentencepiece
print("Dependencies installed")

Mounted at /content/drive
Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 107.6 MB/s eta 0:00:00
Dependencies installed


In [ ]:
# import os
# import requests

# MODEL_DIR = '/content/drive/MyDrive/phi3-merged'
# base_url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct/raw/main/"

# files_to_download = [
#     "modeling_phi3.py"
# ]

# for filename in files_to_download:
#     try:
#         url = base_url + filename
#         response = requests.get(url)
#         if response.status_code == 200:
#             filepath = os.path.join(MODEL_DIR, filename)
#             with open(filepath, 'wb') as f:
#                 f.write(response.content)
#             print(f"Downloaded {filename}")
#         else:
#             print(f"Could not download {filename}")
#     except Exception as e:
#         print(f"Error downloading {filename}: {e}")

Downloaded modeling_phi3.py

 Files downloaded.


In [ ]:
!pip install -q onnxruntime-genai onnx_ir
!pip install --upgrade transformers tokenizers safetensors

import os
import glob

MODEL_DIR = '/content/drive/MyDrive/phi3-merged'
OUTPUT_DIR = '/content/drive/MyDrive/phi3-onnx-output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Converting Phi-3 to ONNX using onnxruntime-genai builder...")
print(f"Input: {MODEL_DIR}")
print(f"Output: {OUTPUT_DIR}\n")

!python -m onnxruntime_genai.models.builder \
    -m "{MODEL_DIR}" \
    -o "{OUTPUT_DIR}" \
    -p int4 \
    -e cuda \
    --extra_options trust_remote_code=True

print("\nChecking output files...")
onnx_files = glob.glob(f"{OUTPUT_DIR}/**/*.onnx*", recursive=True)
if onnx_files:
    print(f"✅ SUCCESS! Created {len(onnx_files)} ONNX related files.")
    for f in onnx_files:
        size_mb = os.path.getsize(f) / (1024**2)
        print(f"  - {os.path.basename(f)}: {size_mb:.2f} MB")
else:
    print("❌ No ONNX files found. Conversion might have failed.")

Converting Phi-3 to ONNX using onnxruntime-genai builder...
Input: /content/drive/MyDrive/phi3-merged
Output: /content/drive/MyDrive/phi3-onnx-output

2025-11-27 11:39:09,631 numexpr.utils [INFO] - NumExpr defaulting to 12 threads.
Valid precision + execution provider combinations are: FP32 CPU, FP32 CUDA, FP16 CUDA, FP16 DML, BF16 CUDA, FP16 TRT-RTX, INT4 CPU, INT4 CUDA, INT4 DML, INT4 WebGPU
Extra options: {'trust_remote_code': 'True'}
GroupQueryAttention (GQA) is used in this model.
2025-11-27 11:39:13.319813: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 11:39:13.337550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000

In [ ]:
import os
files = os.listdir(OUTPUT_DIR)
for f in sorted(files):
    print(f"  - {f}")

  - added_tokens.json
  - chat_template.jinja
  - genai_config.json
  - model.onnx
  - model.onnx.data
  - special_tokens_map.json
  - tokenizer.json
  - tokenizer.model
  - tokenizer_config.json


In [ ]:
!pip uninstall -y onnxruntime onnxruntime-genai
!pip install onnxruntime-genai-cuda

Found existing installation: onnxruntime 1.23.2
Uninstalling onnxruntime-1.23.2:
  Successfully uninstalled onnxruntime-1.23.2
Found existing installation: onnxruntime-genai 0.11.2
Uninstalling onnxruntime-genai-0.11.2:
  Successfully uninstalled onnxruntime-genai-0.11.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.0 MB/s eta 0:00:00


In [ ]:
import traceback
import onnxruntime_genai as og
import numpy as np

OUTPUT_DIR = '/content/drive/MyDrive/phi3-onnx-output'

print("\n" + "="*70)
print("Test Causal LM (Phi-3) Inference for Summarization")
print("="*70)

text_to_summarize = (
    "A recent evaluation of the company’s customer support operations found that while response times have improved significantly, issue resolution rates remain inconsistent across regions. The report attributes these discrepancies to uneven staffing levels and varying degrees of product knowledge among support teams. Leadership is considering a revised training program and standardized workflow guidelines to enhance consistency and service quality."
)

formatted_prompt = f"Summarize the following article concisely: {text_to_summarize}"
prompt = f"<|user|>\n{formatted_prompt}<|end|>\n<|assistant|>\n"

try:
    print("Loading ONNX GenAI model...")
    model = og.Model(OUTPUT_DIR)
    tokenizer = og.Tokenizer(model)
    input_tokens = tokenizer.encode(prompt)
    input_tokens_np = np.array(input_tokens, dtype=np.int32)
    prompt_length = len(input_tokens)
    params = og.GeneratorParams(model)
    params.set_search_options(
        max_length=2048,
        num_beams=4,
        do_sample=False,
    )
    generator = og.Generator(model, params)
    generator.append_tokens(input_tokens_np)

    print("Generating...")
    while not generator.is_done():
        generator.generate_next_token()
    all_tokens = generator.get_sequence(0)
    new_tokens = all_tokens[prompt_length:]
    summary_text = tokenizer.decode(new_tokens)

    print(f"\nSummarization successful!")
    print(f"Original Text Length: {len(text_to_summarize)} chars")
    print(f"Summary:\n{summary_text.strip()}")

except Exception as e:
    print(f"Test failed: {e}")
    traceback.print_exc()


Test Causal LM (Phi-3) Inference for Summarization
Loading ONNX GenAI model...
Generating...

Summarization successful!
Original Text Length: 448 chars
Summary:
Evaluation of customer support operations reveals inconsistent issue resolution rates across regions.
Discrepancies attributed to uneven staffing and varying degrees of product knowledge.
Leadership considering revised training program and standardized workflow guidelines .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!zip -r /content/drive/MyDrive/phi3_mini_4k_onnx_q4.zip /content/drive/MyDrive/phi3-onnx-output

Mounted at /content/drive
  adding: content/drive/MyDrive/phi3-onnx-output/ (stored 0%)
  adding: content/drive/MyDrive/phi3-onnx-output/model.onnx.data